<a href="https://colab.research.google.com/github/nisbenz/Mini-Transformer/blob/main/Residual%26Norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files

uploaded = files.upload()

Saving simplemath_100k_balanced.csv to simplemath_100k_balanced.csv


In [2]:
import pandas as pd
import io
file_name = 'simplemath_100k_balanced.csv'
ds = pd.read_csv(file_name)
print(f"Successfully loaded '{file_name}' into a DataFrame.")
display(ds.head())

Successfully loaded 'simplemath_100k_balanced.csv' into a DataFrame.


,problem,answer
0,2790 + 6698 =,9488
1,8262 + 3848 =,12110
2,1628 + 132 =,1760
3,9925 + 7169 =,17094
4,3023 + 6929 =,9952


In [3]:
import pandas as pd
X_train = pd.Series(ds["problem"])
X_train = X_train.str.ljust(13, ' ')
raw_chars = sorted(list(set(''.join(X_train))))
vocab =  raw_chars
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
print(f"Vocabulary Size: {len(vocab)}")
print("Character Map:", stoi)
def encode_problem(problem_str):
    return [stoi[c] for c in problem_str]
X_train_encoded = [encode_problem(x) for x in X_train]
print("\nExample Output:")
print(f"Original: '{X_train[0]}'")
print(f"Encoded:  {X_train_encoded[0]}")




Vocabulary Size: 16
Character Map: {' ': 0, '+': 1, '-': 2, '/': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '=': 14, 'x': 15}

Example Output:
Original: '2790 + 6698 ='
Encoded:  [6, 11, 13, 4, 0, 1, 0, 10, 10, 13, 12, 0, 14]


In [4]:
import torch
import torch.nn as nn
import math
VOCAB_SIZE = 19
D_MODEL = 128
MAX_LEN = 13
embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=D_MODEL, padding_idx=0)
X_train_tensor = torch.tensor(X_train_encoded, dtype=torch.long)
X_embedded = embedding(X_train_tensor)
print(f"Shape of X_embedded: {X_embedded.shape}")
print(f"Value of X_embedded[0][0]: {X_embedded[0][0]}")

Shape of X_embedded: torch.Size([75000, 13, 128])
Value of X_embedded[0][0]: tensor([ 0.9728,  1.9068, -1.4953, -0.6591, -0.6159, -0.2532, -0.7708,  0.2867,
        -0.0929, -0.1603, -0.1604,  2.3611,  1.6310,  0.7191, -0.6306, -1.0871,
        -0.9088, -0.4935,  1.0571, -0.6832,  0.3481,  0.9360,  0.4868,  0.6432,
        -1.6455, -0.3670,  0.2733, -1.0497,  0.1177, -1.1325, -1.5873,  2.7323,
        -1.0168,  0.0586,  2.0390,  0.0376,  0.4781, -1.1733, -2.1088, -0.8672,
         0.0326, -1.9751,  1.3810,  0.3266, -0.3751, -0.8352,  1.0141,  0.5758,
         0.9581,  0.1533, -0.3010, -0.1552,  0.2966,  0.3321, -0.3321,  1.8166,
        -2.1340, -0.4539, -0.4042,  0.4358, -0.1139,  0.0112, -0.8319, -0.6471,
        -0.4633, -0.7848, -0.2932,  1.3117, -0.6952,  1.9398, -0.4224, -0.0963,
         0.1550, -0.0198,  0.8244,  1.2026, -0.4140, -0.1186,  0.2053, -2.3690,
         0.6434, -0.5750,  1.7986,  0.4319, -0.8994, -0.8548, -0.4901, -1.0480,
         1.2557, -0.3161, -0.4428,  0.9644,

In [5]:
import numpy as np
import tensorflow as tf

def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)


seq_len = X_embedded.shape[1]
d_model = X_embedded.shape[2]

pos_enc_tf = positional_encoding(seq_len, d_model)

pos_enc_torch = torch.from_numpy(pos_enc_tf.numpy()).to(X_embedded.device)

X_final = X_embedded + pos_enc_torch

print(f"Final Input Shape: {X_final.shape}")
print(f"First value after position added: {X_final[0][0]}")


Final Input Shape: torch.Size([75000, 13, 128])
First value after position added: tensor([ 9.7276e-01,  2.9068e+00, -1.4953e+00,  3.4090e-01, -6.1589e-01,
         7.4676e-01, -7.7084e-01,  1.2867e+00, -9.2873e-02,  8.3972e-01,
        -1.6036e-01,  3.3611e+00,  1.6310e+00,  1.7191e+00, -6.3056e-01,
        -8.7125e-02, -9.0875e-01,  5.0649e-01,  1.0571e+00,  3.1680e-01,
         3.4810e-01,  1.9360e+00,  4.8676e-01,  1.6432e+00, -1.6455e+00,
         6.3296e-01,  2.7334e-01, -4.9729e-02,  1.1765e-01, -1.3252e-01,
        -1.5873e+00,  3.7323e+00, -1.0168e+00,  1.0586e+00,  2.0390e+00,
         1.0376e+00,  4.7808e-01, -1.7332e-01, -2.1088e+00,  1.3276e-01,
         3.2632e-02, -9.7506e-01,  1.3810e+00,  1.3266e+00, -3.7507e-01,
         1.6485e-01,  1.0141e+00,  1.5758e+00,  9.5813e-01,  1.1533e+00,
        -3.0100e-01,  8.4479e-01,  2.9658e-01,  1.3321e+00, -3.3207e-01,
         2.8166e+00, -2.1340e+00,  5.4610e-01, -4.0417e-01,  1.4358e+00,
        -1.1392e-01,  1.0112e+00, -8.3189e

In [6]:
import torch
import math

BATCH_SIZE = X_final.shape[0]
SEQ_LEN    = X_final.shape[1]
D_MODEL    = 128
N_HEAD     = 4
HEAD_DIM   = D_MODEL // N_HEAD
W_q = nn.Linear(D_MODEL, D_MODEL)
W_k = nn.Linear(D_MODEL, D_MODEL)
W_v = nn.Linear(D_MODEL, D_MODEL)
Q = W_q(X_final)
K = W_k(X_final)
V = W_v(X_final)
Q = Q.view(BATCH_SIZE, SEQ_LEN, N_HEAD, HEAD_DIM)
K = K.view(BATCH_SIZE, SEQ_LEN, N_HEAD, HEAD_DIM)
V = V.view(BATCH_SIZE, SEQ_LEN, N_HEAD, HEAD_DIM)
Q = Q.transpose(1, 2)
K = K.transpose(1, 2)
V = V.transpose(1, 2)
print("Shape per Head:", Q.shape)
def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = Q.size(-1)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    attn_weights = torch.softmax(scores, dim=-1)
    output = torch.matmul(attn_weights, V)

    return output, attn_weights

values, attention_weights = scaled_dot_product_attention(Q, K, V)
print("Output Shape:", values.shape)

Shape per Head: torch.Size([75000, 4, 13, 32])
Output Shape: torch.Size([75000, 4, 13, 32])


In [7]:
values = values.transpose(1, 2)
values = values.contiguous()
concat_output = values.view(BATCH_SIZE, SEQ_LEN, D_MODEL)
print("Merged Shape:", concat_output.shape)
W_o = nn.Linear(D_MODEL, D_MODEL)
final_output = W_o(concat_output)
print("Final Attention Output:",final_output.shape)
dropout_layer = nn.Dropout(p=0.1)
Z_dropped = dropout_layer(final_output)
print("Shape after Dropout:", Z_dropped.shape)
calc = Z_dropped+ X_final
layer_norm = nn.LayerNorm(normalized_shape=D_MODEL)
out1 = layer_norm(calc)
print("Shape after Add & Norm:", out1.shape)


Merged Shape: torch.Size([75000, 13, 128])
Final Attention Output: torch.Size([75000, 13, 128])
Shape after Dropout: torch.Size([75000, 13, 128])
Shape after Add & Norm: torch.Size([75000, 13, 128])
